In [34]:
# Regressive Model Template
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import metrics
import os
from importlib.machinery import SourceFileLoader
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
helpers = SourceFileLoader("helpers", "../helpers.py").load_module()
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint


# read in all data from the auto table
fileData = os.path.join('../data', 'auto-mpg.csv')
df = pd.read_csv(fileData,na_values=['NA','?'])

cars = df['name']
df.drop('name',1,inplace=True)
helpers.missing_median(df, 'horsepower')

helpers.encode_text_dummy(df, 'origin')

x,y = helpers.to_xy(df,"mpg")


# Split into train/test
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.25, random_state=45)

model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))

model.add(Dense(1))  # 1 output neuron 





model.compile(loss='mean_squared_error', optimizer='adam')

# stops when data is flattening out
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.fit(x_train,y_train, validation_data=(x_test,y_test),callbacks=[monitor],verbose=0,epochs=1000)


pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(pred, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result


Epoch 00065: early stopping
Final score (RMSE): 5.276342868804932


,ground_truth,predicted
0,25.100000,28.839310
1,32.299999,30.650375
2,18.500000,23.223911
3,34.000000,30.743999
4,30.900000,29.001377
...,...,...
95,22.000000,25.324875
96,23.900000,22.433315
97,13.000000,13.775752
98,24.000000,27.021099


In [45]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import scikitplot as skplt

path = "../data/"
preprocess = True

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# create feature vector
helpers.missing_median(df, 'horsepower')
helpers.encode_text_dummy(df, 'origin')
df.drop('name',1,inplace=True)

if preprocess:
    helpers.encode_numeric_zscore(df, 'horsepower')
    helpers.encode_numeric_zscore(df, 'weight')
    helpers.encode_numeric_zscore(df, 'cylinders')
    helpers.encode_numeric_zscore(df, 'displacement')
    helpers.encode_numeric_zscore(df, 'acceleration')
    helpers.encode_numeric_zscore(df, 'year')

# Encode to a 2D matrix for training
x,y = helpers.to_xy(df,'mpg')



# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=0,epochs=1000)

pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(pred, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result

skplt.metrics.plot_cumulative_gain(y_test, pred)

ModuleNotFoundError: No module named 'scikitplot'